In [0]:
from google.colab import drive
drive.mount('/content/drive')
!pip install tensorly
!pip install pmdarima
%cd /content/drive/My Drive/Colab Notebooks/Group30_Final_Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Group30_Final_Project


In [0]:
import pandas as pd
import numpy as np

m5_sales =pd.read_csv('input/ M5DataSet/sales_train_validation.csv',
               sep=',' , header=None)
m5_sales = np.delete(m5_sales.values, 0, 0)
m5_sales = np.delete(m5_sales, (0,1,2,3,4,5), 1)
m5_sales = np.array(list(m5_sales[:,:]), dtype=np.float)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,

In [0]:
def RMSSE(seria, y_pred, h):

  t_1 = np.sum(np.square(seria[-h:] - y_pred))
  t_2 = np.sum(np.square(seria[1:] - seria[0:-1]))
  t_2 = t_2 / (len(seria)-1) 

  rmsse = np.sqrt(t_1 / t_2 / h)

  return(rmsse)

def WRMSSE(data, y_pred, h, w):

  wrm = 0

  for i in range(data.shape[0]):

    wrm += w[i] * RMSSE(data[i], y_pred[i], h)

  return(wrm)


In [0]:
from BHT_ARIMA.util.utility import get_acc, nrmse

def naiv(data, h):

  y_naiv = np.reshape(data[..., -h:-h+1], (data.shape[0], 1))
  y_pred = y_naiv

  y_true = data[..., -h:]

  for i in range(h-1):

    y_pred = np.hstack((y_pred, y_naiv))
  
  w = np.ones(data.shape[0]) / data.shape[0] 
  
  stat = {}
  stat['acc'] = get_acc(y_pred, y_true)
  stat['nrmse'] = nrmse(y_pred, y_true)
  stat['wrmsse'] =  WRMSSE(data, y_pred, h, w)

  return(stat)

In [0]:
import time

from BHT_ARIMA import BHTARIMA
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_BHTARIMA_m5(data, param):

      p = param['p'] # p-order
      d = param['d'] # d-order
      q = param['q'] # q-order
      taus = param['taus'] # MDT-rank
      Rs = param['Rs'] # tucker decomposition ranks
      k =  param['k'] # iterations
      tol = param['tol'] # stop criterion
      Us_mode = param['Us_mode'] # orthogonality mod
      h = param['h']

      result_full = np.zeros([data.shape[0], h])

      ts = data[..., 0:-h].copy()

      for i in range(h):

        model = BHTARIMA(ts, p, d, q, taus, Rs, k, tol, verbose=0, Us_mode=Us_mode)
        result, _ = model.run()
        result_full[..., i] = np.rint(result[..., -1])
        ts = np.hstack((ts, np.reshape(ts[..., -1], (ts.shape[0], 1))))

      label = data[..., -h:]

      w = np.ones(data.shape[0]) / data.shape[0]

      stat = {}
      stat['acc'] = get_acc(result_full, label)
      stat['nrmse'] = nrmse(result_full, label)
      stat['wrmsse'] =  WRMSSE(data, result_full, h, w)

      return(stat)


In [0]:
import numpy as np
import time

import pmdarima as pm

from BHT_ARIMA.util.utility import get_acc, nrmse

def run_ARIMA_m5(data, h):


      result_full = np.zeros([data.shape[0], h])
      y = data[..., :-h].copy()

      for j in range(y.shape[0]):
        
        model = pm.AutoARIMA(seasonal=True, m=7, suppress_warnings=True,
                             start_p = 1, max_p = 3, max_d = 2, start_q = 1,
                             max_q = 2, 
                             trace=False,error_action="ignore")

        result = np.rint(model.fit_predict(y[j], n_periods=h))
        result_full[j] = result

      true_value = data[..., -h:]

      w = np.ones(data.shape[0]) / data.shape[0]


      stat = {}
      stat['acc'] = get_acc(result_full, true_value)
      stat['nrmse'] = nrmse(result_full, true_value)
      stat['wrmsse'] =  WRMSSE(data, result_full, h, w)


      return(stat)

In [0]:
import numpy as np
import pandas as pd

from fbprophet import Prophet
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_Prophet_m5(data, h):


      start = '2011-01-29'
      freq = 'D'

      time_st = pd.period_range(start=start, periods=data.shape[-1], freq=freq).to_timestamp()
 
      DF_ts = pd.DataFrame()
      DF_ts['ds'] = time_st.copy()
      

      result_full = np.zeros([data.shape[0], h])
        
      for j in range(data.shape[0]):

        DF_ts['y'] = data[j].copy()

        model = Prophet(daily_seasonality=True)
        model.fit(DF_ts[:-h])
        result = np.rint(model.predict(DF_ts[['ds']][-h:])['yhat'].values)
        result_full[j] = result

      true_value = data[..., -h:]

      w = np.ones(data.shape[0]) / data.shape[0]


      stat = {}
      stat['acc'] = get_acc(result_full, true_value)
      stat['nrmse'] = nrmse(result_full, true_value)
      stat['wrmsse'] =  WRMSSE(data, result_full, h, w)



      return(stat)

In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [0]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 38.0 gigabytes of available RAM

You are using a high-RAM runtime!


In [0]:
param_m5 = {
'p' : 3,
'd' : 1,
'q' : 1,
'taus' :  [m5_sales.shape[0], 20],
'Rs' : [5, 5],
'k' :  15,
'tol' : 0.001,
'Us_mode' : 4,
'h' : 28,
}
run_BHTARIMA_m5(m5_sales, param_m5)

{'acc': 0.2752721195273167,
 'nrmse': 2.2085628535522925,
 'wrmsse': 1.14019246697204}

In [0]:
h = 28
run_ARIMA_m5(m5_sales, h)

{'acc': 0.6666666666666666,
 'nrmse': 0.8333333333333334,
 'wrmsse': 1.1766494731317774}

In [0]:
h = 28
run_Prophet_m5(m5_sales, h)

{'acc': 0.3450321431832949,
 'nrmse': 1.7764155933173502,
 'wrmsse': 0.8383773282560876}